In [1]:
!pip install sentence-transformers scikit-learn



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import pickle

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


/opt/homebrew/Cellar/jupyterlab/4.5.1/libexec/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd

df = pd.read_csv("train.csv")

df = df.dropna(subset=['question1', 'question2', 'is_duplicate'])
df['is_duplicate'] = df['is_duplicate'].astype(int)

# Balanced sampling (important)
df_dup = df[df['is_duplicate'] == 1].sample(60000, random_state=42)
df_non = df[df['is_duplicate'] == 0].sample(60000, random_state=42)

df = pd.concat([df_dup, df_non])
df = df.sample(frac=1, random_state=42)  # shuffle

print(df['is_duplicate'].value_counts())



is_duplicate
0    60000
1    60000
Name: count, dtype: int64


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')




Loading weights: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 971.82it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [6]:
q1_embeddings = model.encode(
    df['question1'].tolist(),
    batch_size=64,
    show_progress_bar=True
)

q2_embeddings = model.encode(
    df['question2'].tolist(),
    batch_size=64,
    show_progress_bar=True
)


Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1875/1875 [01:23<00:00, 22.37it/s]


In [7]:

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# cosine similarity
cos_sim = np.array([
    cosine_similarity([q1_embeddings[i]], [q2_embeddings[i]])[0][0]
    for i in range(len(q1_embeddings))
]).reshape(-1, 1)

# difference
abs_diff = np.abs(q1_embeddings - q2_embeddings)

# multiplication
mult = q1_embeddings * q2_embeddings

# FINAL STRONG FEATURE MATRIX
X = np.hstack((
    q1_embeddings,
    q2_embeddings,
    abs_diff,
    mult,
    cos_sim
))

y = df['is_duplicate'].values

print("X shape:", X.shape)
print("y shape:", y.shape)







X shape: (120000, 1537)
y shape: (120000,)


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [9]:

pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [10]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(x_train, y_train)





[LightGBM] [Info] Number of positive: 48000, number of negative: 48000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.779744 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391425
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 1535
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,boosting_type,'gbdt'
,num_leaves,64
,max_depth,8
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [11]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))



/opt/homebrew/Cellar/jupyterlab/4.5.1/libexec/lib/python3.14/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.8569166666666667


In [12]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully as model.pkl")


Model saved successfully as model.pkl
